In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste import paste_align, paste_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2024-07-02 00:51:11.983703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 00:51:12.147200: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-02 00:51:12.147237: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
20

In [1]:
data_folder = "./results/split_slices/"
results_folder = "./results/PASTE/"
figures_folder = "./results/figures/PASTE"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [2]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(59)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"slice_{i}.h5ad")))

100%|████████████████████████| 59/59 [03:44<00:00,  3.81s/it]


In [3]:
spatial_key = "spatial"
anno_key = "class"
palette = slices[0].uns[f"{anno_key}_palette"]

In [4]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [5]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [ ]:
## Perform the PASTE alignment
sampling_num = 10000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = paste_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="2",
        verbose=False,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                 | 0/58 [00:00<?, ?it/s]

gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
  2%|▍                     | 1/58 [02:45<2:36:54, 165.17s/it]

gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
  3%|▊                     | 2/58 [05:21<2:29:24, 160.07s/it]

...

/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 91%|███████████████████▏ | 53/58 [2:11:11<12:56, 155.34s/it]

gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
 93%|███████████████████▌ | 54/58 [2:15:19<12:13, 183.32s/it]

gpu is available, using gpu.


/home/ylu/anaconda3/envs/Spateo/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)
